In [124]:
import requests
from bs4 import BeautifulSoup
import json

In [125]:
session = requests.session()

In [126]:
r = session.get('https://www.instagram.com/fearythanyarat/')

In [127]:
response = BeautifulSoup(r.text, 'html.parser')

In [128]:
scripts = response.find_all('script')

In [129]:
find_shared_data = lambda x: x.text.find('window._sharedData') != -1

In [130]:
shared_data_script = next(filter(find_shared_data, scripts), None)

In [131]:
shared_data = json.loads(shared_data_script.text[21:-1])

In [132]:
edges = shared_data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['edges']

In [143]:
edges[0]['node'].keys()

dict_keys(['__typename', 'id', 'edge_media_to_caption', 'shortcode', 'edge_media_to_comment', 'comments_disabled', 'taken_at_timestamp', 'dimensions', 'display_url', 'edge_liked_by', 'edge_media_preview_like', 'location', 'gating_info', 'fact_check_overall_rating', 'fact_check_information', 'media_preview', 'owner', 'thumbnail_src', 'thumbnail_resources', 'is_video', 'accessibility_caption'])

In [10]:
user_id = shared_data['entry_data']['ProfilePage'][0]['graphql']['user']['id']
user_count = shared_data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['count']
end_cursor = shared_data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['page_info']['end_cursor']
user_rhx_gis = shared_data.get('rhx_gis', None)
# user_id, user_count, end_cursor, user_rhx_gis

In [11]:
# QUERY_HASH = '2c5d4d8b70cad329c4a6ebe3abb6eedd'
# query_params = {
#     'query_hash': QUERY_HASH,
#     'variable': json.dumps({
#         'id': user_id,
#         'first': 12,
#         'end_cursor': end_cursor,
#     }),
# }
# API_URL = 'https://www.instagram.com/graphql/query'
# query_response = session.get(API_URL, params=query_params)

In [144]:
API_URL = 'https://instagram.com/graphql/query/?' + \
    'query_hash={query_hash}&' + \
    'variables=%7B"id"%3A"{id}"%2C"first"%3A{first}%2C"after"%3A"{after}"%7D'
QUERY_HASH = '2c5d4d8b70cad329c4a6ebe3abb6eedd'
api_url = API_URL.format(
    query_hash=QUERY_HASH, id=user_id, first=12, after=end_cursor)
query_response = session.get(api_url)

In [145]:
query_response = json.loads(query_response.text)

In [156]:
query_response['data']['user']['edge_owner_to_timeline_media']['edges'] # ['page_info']['end_cursor'],

[{'node': {'__typename': 'GraphSidecar',
   'id': '2152846740167200053',
   'dimensions': {'height': 1350, 'width': 1080},
   'display_url': 'https://instagram.fmkz1-1.fna.fbcdn.net/vp/a5c200c035d743b0dc81adaed1503265/5E626357/t51.2885-15/e35/p1080x1080/71178324_460182021514424_6700599480799854027_n.jpg?_nc_ht=instagram.fmkz1-1.fna.fbcdn.net&_nc_cat=108',
   'display_resources': [{'src': 'https://instagram.fmkz1-1.fna.fbcdn.net/vp/72f26f654f82de2b03c4c057e3e94799/5E6293B2/t51.2885-15/sh0.08/e35/p640x640/71178324_460182021514424_6700599480799854027_n.jpg?_nc_ht=instagram.fmkz1-1.fna.fbcdn.net&_nc_cat=108',
     'config_width': 640,
     'config_height': 800},
    {'src': 'https://instagram.fmkz1-1.fna.fbcdn.net/vp/a7371590d49536a485c871d9f4bbbe93/5E5E1FB2/t51.2885-15/sh0.08/e35/p750x750/71178324_460182021514424_6700599480799854027_n.jpg?_nc_ht=instagram.fmkz1-1.fna.fbcdn.net&_nc_cat=108',
     'config_width': 750,
     'config_height': 937},
    {'src': 'https://instagram.fmkz1-1.fna.fb

In [93]:
# query_response['data']['user']['edge_owner_to_timeline_media']['edges'][0]['node']['shortcode']

In [105]:
def get_short_code(edge):
    edge_time_taken = edge['node']['taken_at_timestamp']
    edge_type = edge['node']['__typename']
    short_code = edge['node']['shortcode']
    return short_code, edge_type

get_short_code(query_response['data']['user']['edge_owner_to_timeline_media']['edges'][6])

('B3Lh_BfhO8s', 'GraphSidecar')

In [99]:
def get_post_by_shortcode(short_code):
    API_URL = 'https://instagram.com/graphql/query/?' + \
    'query_hash={query_hash}&' + \
    'variables=%7B"shortcode"%3A"{short_code}"%2C"child_comment_count"%3A{child_comment_count}%2C"fetch_comment_count"%3A{fetch_comment_count}%2C"parent_comment_count"%3A{parent_comment_count}%2C"has_threaded_comments"%3A{has_threaded_comments}%7D'

    api_url = API_URL.format(
        query_hash='fead941d698dc1160a298ba7bec277ac',
        short_code=short_code,
        child_comment_count=3,
        fetch_comment_count=40,
        parent_comment_count=24,
        has_threaded_comments='true',
    )

    query_response = session.get(api_url)
    
    return json.loads(query_response.text)

In [113]:
def download_node(node):
    if node['__typename'] == 'GraphImage':
        print(node['display_url'])
        
    if node['__typename'] == 'GraphVideo':
        print(node['video_url'])
    
    if node['__typename'] == 'GraphSidecar':
        for edge in node['edge_sidecar_to_children']['edges']:
            download_node(edge['node'])

In [157]:
resp = get_post_by_shortcode('B3Lh_BfhO8s')

In [158]:
resp['data']['shortcode_media'].keys()

dict_keys(['__typename', 'id', 'shortcode', 'dimensions', 'gating_info', 'fact_check_overall_rating', 'fact_check_information', 'media_preview', 'display_url', 'display_resources', 'is_video', 'tracking_token', 'edge_media_to_tagged_user', 'edge_media_to_caption', 'caption_is_edited', 'has_ranked_comments', 'edge_media_to_parent_comment', 'edge_media_preview_comment', 'comments_disabled', 'taken_at_timestamp', 'edge_media_preview_like', 'edge_media_to_sponsor_user', 'location', 'viewer_has_liked', 'viewer_has_saved', 'viewer_has_saved_to_collection', 'viewer_in_photo_of_you', 'viewer_can_reshare', 'owner', 'is_ad', 'edge_web_media_to_related_media', 'edge_sidecar_to_children'])

In [159]:
resp['data']['shortcode_media']['taken_at_timestamp']

1570157492

In [120]:
node = resp['data']['shortcode_media']

In [121]:
download_node(node)

https://instagram.fmkz1-1.fna.fbcdn.net/vp/431be02e7b462ff320f4963f39e2ba0f/5E47BACD/t51.2885-15/e35/p1080x1080/72344087_767119600392560_94310000090527221_n.jpg?_nc_ht=instagram.fmkz1-1.fna.fbcdn.net&_nc_cat=108
https://instagram.fmkz1-1.fna.fbcdn.net/vp/32bbac68183bb6a4e7d8f43d53447e48/5E61BD83/t51.2885-15/e35/p1080x1080/71548123_200575094269374_6001961081335071383_n.jpg?_nc_ht=instagram.fmkz1-1.fna.fbcdn.net&_nc_cat=106
https://instagram.fmkz1-1.fna.fbcdn.net/vp/949016c4aab2dddbe48a00a043d4909b/5E6571EB/t51.2885-15/e35/p1080x1080/70848618_399946217334350_8627530699969740949_n.jpg?_nc_ht=instagram.fmkz1-1.fna.fbcdn.net&_nc_cat=102
https://instagram.fmkz1-1.fna.fbcdn.net/vp/2242fb11f325ea2bdd676783415cc8d9/5E44E610/t51.2885-15/e35/p1080x1080/69897049_142536150331792_3058184831245567766_n.jpg?_nc_ht=instagram.fmkz1-1.fna.fbcdn.net&_nc_cat=104
https://instagram.fmkz1-1.fna.fbcdn.net/vp/98161bb276722e3f70c351bb25d8118c/5E48190C/t51.2885-15/e35/p1080x1080/70393935_154932552277188_649784015

In [74]:
resp = get_post_by_shortcode('B3b67I2BCZx')

In [83]:
resp['data']['shortcode_media']['__typename'], resp['data']['shortcode_media']['video_url']

('GraphVideo',
 'https://instagram.fmkz1-1.fna.fbcdn.net/vp/bc9e462f70e89702395175e8491796f7/5DC8DC3F/t50.2886-16/70041034_2453158854707544_5597102290941404757_n.mp4?_nc_ht=instagram.fmkz1-1.fna.fbcdn.net&_nc_cat=100')

In [100]:
resp = get_post_by_shortcode('B3TgsXDh3w3')

In [103]:
resp['data']['shortcode_media']['__typename'], resp['data']['shortcode_media']['display_url']

('GraphImage',
 'https://instagram.fmkz1-1.fna.fbcdn.net/vp/4edef4587aabc774662e069e02549037/5E4399BA/t51.2885-15/e35/p1080x1080/70124914_1943038452466412_1780564089840610789_n.jpg?_nc_ht=instagram.fmkz1-1.fna.fbcdn.net&_nc_cat=108')

In [106]:
resp = get_post_by_shortcode('B3Lh_BfhO8s')

In [112]:
resp['data']['shortcode_media']['__typename'], resp['data']['shortcode_media']['edge_sidecar_to_children']['edges']

('GraphSidecar',
 [{'node': {'__typename': 'GraphImage',
    'id': '2146959111244493841',
    'shortcode': 'B3Lh-_FBDQR',
    'dimensions': {'height': 1350, 'width': 1080},
    'gating_info': None,
    'fact_check_overall_rating': None,
    'fact_check_information': None,
    'media_preview': 'ACEqoBXijEh6PkKD7HBOP85pA0i/K/fsRU1wpiVefuE8fTnP0NaaKJ0DPja4zjHI/H9ag0SM94UVPM5IyBgH1q7p4HlkqCBk9TntVaKMMTHyEYjGTjJ7Yz+oFaiR+Sm3AXgkDv8Aj70CZjeb/tf+PGiovsz/AN0frRQBd1AoRnByB+tZtndeU4WTJQH16fX1A/SrU8JWPLEt6e1ZYU9uc0w9DtAVdSCAwI6dc1n3DfZgHX54ycYPVfoeuPY5xUdgzLENxxgnt0HYfQH/AOtUGqK+0N/yzB5x0B7H8fyBpLsN7XH/ANpj+7RWHuoq7EHQ3YPlcetZMcBkfA4zya3bj7oqDuPpUgQncE2pwBxzzTI52SMxyjduBBI75+tOkPFVmoKM77M3rRVyimKx/9k=',
    'display_url': 'https://instagram.fmkz1-1.fna.fbcdn.net/vp/431be02e7b462ff320f4963f39e2ba0f/5E47BACD/t51.2885-15/e35/p1080x1080/72344087_767119600392560_94310000090527221_n.jpg?_nc_ht=instagram.fmkz1-1.fna.fbcdn.net&_nc_cat=108',
    'display_resources': [{'src': 'https://instagram.fmkz1-1.fna

In [49]:
API_URL = 'https://instagram.com/graphql/query/?' + \
    'query_hash={query_hash}&' + \
    'variables=%7B"shortcode"%3A"{short_code}"%2C"child_comment_count"%3A{child_comment_count}%2C"fetch_comment_count"%3A{fetch_comment_count}%2C"parent_comment_count"%3A{parent_comment_count}%2C"has_threaded_comments"%3A{has_threaded_comments}%7D'

api_url = API_URL.format(
    query_hash='fead941d698dc1160a298ba7bec277ac',
    short_code='B1-cdlXhxdp',
    child_comment_count=3,
    fetch_comment_count=40,
    parent_comment_count=24,
    has_threaded_comments='true',
)

query_response = session.get(api_url)